In [4]:
import boto3
import os
from dotenv import load_dotenv


In [6]:
load_dotenv()

session = boto3.Session(aws_access_key_id= os.getenv("aws_access_key_id") , 
                        aws_secret_access_key=os.getenv("aws_secret_access_key"))


In [14]:
# Créer un client S3 avec la région Paris
s3_client = boto3.client('s3', region_name='eu-west-3')

# Nom unique du bucket (doit être globalement unique dans tout AWS)
bucket_name = 'bucket-for-kayak-project-myriam-25'

# Créer le bucket avec la configuration de la région
response = s3_client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'}
)

In [15]:
s3_client = boto3.client('s3')

response = s3_client.list_buckets()

print("Buckets existants :")
for bucket in response['Buckets']:
    print(f"  - {bucket['Name']}")

Buckets existants :
  - bucket-for-kayak-project-myriam-25


In [17]:
file_path = 'data/35_best_cities_weather.csv'  
object_name = '35_best_cities_weather.csv'  # Nom du fichier dans le bucket

# Upload du fichier
s3_client.upload_file(file_path, bucket_name, object_name)

print(f"✅ Fichier '{file_path}' envoyé dans le bucket '{bucket_name}' sous le nom '{object_name}'.")

✅ Fichier 'data/35_best_cities_weather.csv' envoyé dans le bucket 'bucket-for-kayak-project-myriam-25' sous le nom '35_best_cities_weather.csv'.


In [18]:
file_path = 'data/best_cities_hotels.csv'  
object_name = 'best_cities_hotels.csv'  # Nom du fichier dans le bucket

# Upload du fichier
s3_client.upload_file(file_path, bucket_name, object_name)

print(f"✅ Fichier '{file_path}' envoyé dans le bucket '{bucket_name}' sous le nom '{object_name}'.")

✅ Fichier 'data/best_cities_hotels.csv' envoyé dans le bucket 'bucket-for-kayak-project-myriam-25' sous le nom 'best_cities_hotels.csv'.


In [19]:
response = s3_client.list_objects_v2(Bucket=bucket_name)

for obj in response.get('Contents', []):
    print(obj['Key'])

35_best_cities_weather.csv
best_cities_hotels.csv


In [39]:
from sqlalchemy import create_engine, text

load_dotenv()

PASSWORD = os.getenv("PASSWORD")
HOSTNAME = os.getenv("HOSTNAME")
DBNAME = os.getenv("DBNAME")

# Create engine will create a connection between a SQLlite DB and python
engine = create_engine(f"postgresql+psycopg2://postgreskayak:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

import pandas as pd

best_cities_hotels = pd.read_csv("s3://bucket-for-kayak-project-myriam-25/best_cities_hotels.csv", storage_options={"key": os.getenv("aws_access_key_id"), "secret": os.getenv("aws_secret_access_key")})
best_cities_hotels.to_sql("best_cities_hotels", engine, if_exists="replace", index=False)

2025-04-24 22:39:14,342 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-04-24 22:39:14,344 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-24 22:39:14,362 INFO sqlalchemy.engine.Engine select current_schema()
2025-04-24 22:39:14,364 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-24 22:39:14,385 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-04-24 22:39:14,387 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-24 22:39:14,415 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-24 22:39:14,442 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

125

In [38]:
best35_cities_weather = pd.read_csv("s3://bucket-for-kayak-project-myriam-25/35_best_cities_weather.csv", storage_options={"key": os.getenv("aws_access_key_id"), "secret": os.getenv("aws_secret_access_key")})
best35_cities_weather.to_sql("best35_cities_weather", engine, if_exists="replace", index=False)

2025-04-24 22:35:24,177 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-24 22:35:24,181 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-04-24 22:35:24,185 INFO sqlalchemy.engine.Engine [cached since 652.8s ago] {'table_name': 'best35_cities_weather', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-04-24 22:35:24,209 INFO sqlalchemy.engine.Engine 
CREATE TABLE best35_cities_weather (
	cities TEXT, 
	country TEXT, 
	state TEXT, 
	lat FLOAT(53), 
	lon FLOAT(53), 
	date TEXT, 
	temp FLOAT(53), 
	feels_like FL

35